In [1]:
import pandas as pd
from os import listdir
from parsedatetime import parsedatetime
from datetime import datetime
from statistics import mode
# Working directory and date parser
wd = "Z:/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Erin Cobry/Prospective HCL and sleep study"
cal = parsedatetime.Calendar()

In [ ]:
# Get file paths
files = listdir(wd + "./Data raw/CGM")
# Iterate through, clean, and calculate variables
results = {"id":[],"total_sensor_readings":[],"percent_cgm_wear":[],
           "percent_time_under_54":[],"minutes_under_54":[],"num_values_under_54":[]}
for f in files:
    results["id"].append(f)
    # Read in
    path = wd + "/Data raw/CGM/" + f
    df = pd.read_csv(path)
    # Find columns
    time_col = [c for c in df.columns if "times" in c.lower()]
    gluc_col = [c for c in df.columns if "glucose value" in c.lower()]
    # Remove unnecessary columns
    clean = pd.concat([df[time_col],df[gluc_col]],axis = 1)
    clean.columns = ["time","glucose"]
    # Remove rows with missing time
    clean = clean[clean['time'].notnull()]
    # Glucose to numeric replace "high" and "low" with limits of detection
    clean["glucose"].replace("High",400,inplace = True)
    clean["glucose"].replace("Low",40,inplace = True)
    clean["glucose"] = [float(g) for g in clean["glucose"]]
    # Format time
    clean["time"] = clean["time"].str.replace('T',' ')
    clean["time"] = [cal.parse(t)[0] for t in clean["time"]]
    clean["time"] = [datetime(*t[:6]) for t in clean["time"]]
    # Find sampling interval
    s = mode(clean["time"].diff()).total_seconds()
    # Re-index and expand to every 5 minutes
    clean.set_index("time",inplace = True)
    clean = clean[~clean.index.duplicated(keep='first')]
    clean = clean.resample(str(s / 60) + "T").ffill()
    clean.reset_index(inplace = True)
    # Write
    clean.to_csv(wd + "/Data clean/CGM/" + f,index = False)
    # Variables
    # CGM wear metrics
    total_r = clean["glucose"].notna().sum()
    results["total_sensor_readings"].append(total_r)
    results["percent_cgm_wear"].append(round(total_r / clean.shape[0] * 100,2))
    # Hypoglycemia
    results["percent_time_under_54"].append(round(len([g for g in clean["glucose"] if g <= 54]) / total_r * 100,2))
    results["minutes_under_54"].append(len([g for g in clean["glucose"] if g <= 54]) * (s / 60))

In [47]:
results

{'id': ['205_BL_CLARITY_19AUG2020.csv',
  '206_BL_CLARITY_11SEPT2020.csv',
  '206_M3_CLARITY_17JAN2021.csv',
  '206_M6_CLARITY_24MAR2021.csv',
  '209_M3_CLARITY_04FEB2021.csv',
  '209_M6_CLARITY_10MAY2021.csv',
  '209_W2_CLARITY_20OCT2020.csv',
  '213_BL_CLARITY_12NOV2020.csv',
  '214_BL_CLARITY_21NOV2020_2week.csv',
  '215_3M_CLARITY_13MAR2021.csv',
  '215_BL_CLARITY_08NOV2020.csv',
  '225_M3_CLARITY_29MAR2021.csv',
  '228_BL_CLARITY_18JAN2021.csv',
  '228_W2_CLARITY_26JAN2021.csv',
  '229_BL_CLARITY_24FEB2021.csv',
  '230_BL_CLARITY_28FEB2021.csv',
  '230_W2_CLARITY_30MAR2021.csv',
  '232_BL_CLARITY_04MAR2021.csv',
  '232_W2_CLARITY_12MAR2021.csv',
  '234_BL_CLARITY_14MAR2021.csv',
  '236_BL_CLARITY_11MAY2021.csv',
  '241_BL_CLARITY_19APR2021.csv',
  '242_BL_CLARITY_10MAY2021.csv',
  '244_BL_CLARITY_16MAY2021.csv',
  '246_BL_CLARITY_01JUN2021.csv'],
 'total_sensor_readings': [2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  536,
  2015,
  4031,
  2015,
  1770,
  2015,
  1957,
  2015,


In [45]:
clean["glucose"].replace("High",400,inplace = True)
clean["glucose"].replace("Low",40,inplace = True)
[float(g) for g in clean["glucose"]]

[nan,
 207.0,
 205.0,
 203.0,
 201.0,
 201.0,
 199.0,
 200.0,
 200.0,
 203.0,
 204.0,
 205.0,
 207.0,
 208.0,
 209.0,
 208.0,
 212.0,
 212.0,
 212.0,
 211.0,
 211.0,
 211.0,
 210.0,
 206.0,
 208.0,
 208.0,
 211.0,
 211.0,
 211.0,
 211.0,
 212.0,
 214.0,
 212.0,
 209.0,
 210.0,
 207.0,
 201.0,
 201.0,
 204.0,
 207.0,
 215.0,
 227.0,
 228.0,
 222.0,
 216.0,
 216.0,
 219.0,
 221.0,
 223.0,
 223.0,
 224.0,
 225.0,
 225.0,
 222.0,
 219.0,
 218.0,
 219.0,
 217.0,
 210.0,
 211.0,
 210.0,
 210.0,
 209.0,
 205.0,
 202.0,
 194.0,
 193.0,
 195.0,
 199.0,
 202.0,
 204.0,
 204.0,
 204.0,
 205.0,
 194.0,
 195.0,
 196.0,
 197.0,
 199.0,
 200.0,
 210.0,
 214.0,
 216.0,
 217.0,
 216.0,
 214.0,
 211.0,
 202.0,
 201.0,
 202.0,
 207.0,
 207.0,
 203.0,
 198.0,
 191.0,
 185.0,
 184.0,
 176.0,
 179.0,
 181.0,
 180.0,
 178.0,
 177.0,
 184.0,
 188.0,
 190.0,
 190.0,
 191.0,
 190.0,
 191.0,
 192.0,
 195.0,
 194.0,
 195.0,
 196.0,
 194.0,
 192.0,
 188.0,
 188.0,
 186.0,
 183.0,
 177.0,
 169.0,
 161.0,
 157.0,
 1

In [44]:
help(clean["glucose"].replace)

Help on method replace in module pandas.core.series:

replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad') method of pandas.core.series.Series instance
    Replace values given in `to_replace` with `value`.
    
    Values of the Series are replaced with other values dynamically.
    
    This differs from updating with ``.loc`` or ``.iloc``, which require
    you to specify a location to update with some value.
    
    Parameters
    ----------
    to_replace : str, regex, list, dict, Series, int, float, or None
        How to find the values that will be replaced.
    
        * numeric, str or regex:
    
            - numeric: numeric values equal to `to_replace` will be
                replaced with `value`
            - str: string exactly matching `to_replace` will be replaced
                with `value`
            - regex: regexs matching `to_replace` will be replaced with
                `value`
    
        * list of str, regex, or num

In [24]:
clean["glucose"].notna().sum()

2011